<hr>
<center><h1>ECDSA : Elliptic Curve Digital Signature Algorithm</h1></center>
<hr>

In [116]:
import hashlib

## Curve P-256

In [112]:
# a large prime number p
p = 0xffffffff00000001000000000000000000000000ffffffffffffffffffffffff
# the finite field of Z/pZ
K = GF(p)
# equation parameters a and b such as : y^2 = x^3 + ax + b
a = K(0xffffffff00000001000000000000000000000000fffffffffffffffffffffffc)
b = K(0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b)
# Define the EC 
E = EllipticCurve(K, (a, b))
# the generator of the EC
G = E(0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296, 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5)
E.set_order(0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551 * 0x1)
n =  0xffffffff00000000ffffffffffffffffbce6faada7179e84f3b9cac2fc632551

## 1. KeyGen

In [117]:
# Alice choses a random between 1 and o(G) d as private key
# d = int(K.random_element(1, n)) # randomized for large keys

# fixed for brute forcing 
d =  0xffff
# public key Q
Q = d*G

Q, d

((109588801698256994951990571109052358531364605021783910115998727948928103999695 : 82091379372138771455752671260931004134817620445688054187650988778738847793385 : 1),
 65535)

## 2. Sign

In [114]:
# chose k : 1<k<q
k = int(K.random_element(1,n))
# calculate R
R = k*G
# set r = x_R
r = Integer(R[0]) % n

# a given message m 
m = "ECDSA signature".encode('utf-8')
h = hashlib.sha256(m).digest()

# Convert to integer
z = int.from_bytes(h, 'big')

# truncate z to n bits
bit_length = n.nbits()
z = z >> max(0, z.bit_length() - bit_length)

#calculate s = k^(-1)(z+r*d) mod p
k_inv = inverse_mod(Integer(k), n)
s = (k_inv * (z + r * d)) % n

if r == 0 or s == 0:
    raise Error("s and r must not be zero values")
    
signature = (r, s)
signature

(20445690346201560616755990788930287230549996864932208381864755305680321526496,
 87705496794746564434657536838581850357636481058310494614056477766437560886129)

## 3. Verify

In [115]:
# calculate u1 = z*s^(-1) mod n
w = inverse_mod(Integer(s), n)

u1 = (z * w) % n
u2 = (r * w) % n

# calculate P = u1*G + u2*Q
P = u1 * G + u2 * Q

# Verifying is (r) signature[0] == x_P
if Integer(signature[0]) == Integer(P[0])%n:
    print(f"Signature {signature} is valid")
else : 
    print(f"Signature {signature} is not valid !")

Signature (20445690346201560616755990788930287230549996864932208381864755305680321526496, 87705496794746564434657536838581850357636481058310494614056477766437560886129) is valid


<hr>

## Attacker : can brute forcing to find d

In [118]:
c = 100000  # this value is grater than d
for i in range(c + 1):
    P = i * G
    print(f"Checking i={i} | Q={Q}", end="\r", flush=True)
    if P == Q :
        sk = i
        print(f"\nFound private key: {i}")
        break

Checking i=65535 | Q=(109588801698256994951990571109052358531364605021783910115998727948928103999695 : 82091379372138771455752671260931004134817620445688054187650988778738847793385 : 1)
Found private key: 65535
